### <div class="alert alert-success" style="background:#2C3E50;color:white">Basic Transformations and Actions - map, flatMap, reduce and more </div>

##### command to launch pyspark explained below
- in yarn mode
- multi-tenant environment hence passing port number so that there is no conflict with other users
- num-executors set to 2
- disable spark dynamic allocation

In [ ]:
pyspark --master yarn \
 --conf spark.ui.port=21117 \
 --num-executors 2 \
 --conf spark.dynamicAlloction.enabled=false

 <p style="background :#AED6F1"><b>eg 1 - Sum of even numbers converting a collection into RDD</b></p>

In [ ]:
>>> list(range(1, 10))
[1, 2, 3, 4, 5, 6, 7, 8, 9]
>>> l = list(range(1, 10))
>>> type(l)
<type 'list'>
>>> l = list(range(1, 100001))
>>> len(l)
100000

In [ ]:
#converting l into RDD using parallelize function to sum up even numbers
>>> lRDD = sc.parallelize(l)
>>> type(lRDD)
<class 'pyspark.rdd.RDD'>

<p style="background:#F1C40F"><b>NOTE :</b>transformations take 1 RDD as input and they generate another RDD as output. they do not trigger the execution, it will update the DAG (Directed Acyclic Graph) associated with the variable of type RDD.</p> 

In [ ]:
#action on RDD
>>> lRDD.count()
[Stage 0:>                                                          (0 + 0) / 2]20/06/11 02:07:50 WARN TaskSetManager: Stage 0 contains a task of very large size (155 KB). The maximum recommended task size is 100 KB.
100000            

<p style="background:#F1C40F"><b>NOTE :</b>when we perform an action, it will return the values to the driver program and also it triggers the execution of DAG as soon as it is performed.</p>

In [ ]:
>>> lRDD.first()
20/06/11 02:08:17 WARN TaskSetManager: Stage 1 contains a task of very large size (155 KB). The maximum recommended task size is 100 KB.
1
>>> lRDD.take(10)
20/06/11 02:08:30 WARN TaskSetManager: Stage 2 contains a task of very large size (155 KB). The maximum recommended task size is 100 KB.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
#filtering even numbers from lRDD
>>> lEven = lRDD.filter(lambda x: x % 2 == 0)
>>> type(lEven)
<class 'pyspark.rdd.PipelinedRDD'>

In [ ]:
>>> lEven.count()
[Stage 3:>                                                          (0 + 0) / 2]20/06/11 02:14:17 WARN TaskSetManager: Stage 3 contains a task of very large size (155 KB). The maximum recommended task size is 100 KB.
50000  

In [ ]:
#reduce is an action
#reduce returns only 1 value
>>> lEven.reduce(lambda x, y: x + y)
[Stage 4:>                                                          (0 + 0) / 2]20/06/11 02:17:43 WARN TaskSetManager: Stage 4 contains a task of very large size (155 KB). The maximum recommended task size is 100 KB.
2500050000

In [ ]:
>>> lEven.reduce(lambda x, y: x if x < y else y)
20/06/11 02:18:21 WARN TaskSetManager: Stage 5 contains a task of very large size (155 KB). The maximum recommended task size is 100 KB.
2

In [ ]:
>>> lEven.reduce(lambda x, y: x if x > y else y)
20/06/11 02:18:36 WARN TaskSetManager: Stage 6 contains a task of very large size (155 KB). The maximum recommended task size is 100 KB.
100000

In [ ]:
#alternative to above lambda func approach
#package operator in python which has func add

>>> from operator import add
>>> lEven.reduce(add)
20/06/11 02:19:44 WARN TaskSetManager: Stage 7 contains a task of very large size (155 KB). The maximum recommended task size is 100 KB.
2500050000

<br><p style="background:#FFA07A;color:red;border:solid"><b>NOTE :DO NOT USE BELOW COMMAND IN CERTIFICATION</b></p>

In [ ]:
>>> lRDD.collect()
# be extermely careful in using this because it converts
# entire RDD into collection, say 10gb data is there in RDD
# whole data is converted into a collection of type list
# and you run into out of memory issues

 <p style="background :#AED6F1"><b>eg 2 - Word Count Program </b></p>

<p style="background:#F1C40F"><b> 1) flatMap -</b> convert a single record into multiple records based upon the logic. Number of records in output RDD will be greater than in input RDD.<br>
<b> 2) map -</b> apply the transformation on individual records resulting in changed values. Number of records in input and output RDDs will be same.<br>
<b> 3) reduceByKey -</b> generate aggregated result by processing data in input RDD. typically returns 1 value per key irrespective of the number of records in input RDD. reduceByKey is a transformation.</p>

<p style="background:#F1C40F"><b>Problem Statment -</b>
For uniques word in an input file we need to get how many times it is repeated.
<br><b>Design -</b>
<br>- Break each line into words using <b>flatMap</b>. <br>flatMap takes lambda function as argument for which we need to pass logic to break down input record into an array and flatMap's inbuilt logic will return each element in array as record.
<br>- After we break each line into word, we need to convert them into tuples using <b>map</b>, with word as key and 1 as it's value.
<br>- Paired RDD (o/p of above map) can now be passed to <b>reduceByKey</b> and get count of each word.</p>

<p style="background :#d0d5db"><b>eg word count solution using list collection</b>
<br>just for prototyping</p>

In [ ]:
#eg using list collection
>>> l = ["Hello how are you?", "you are welcome", "welcome to xxxxxx"]
>>> l
['Hello how are you?', 'you are welcome', 'welcome to xxxxxx']

In [ ]:
>>> l[0]
'Hello how are you?'
>>> len(l)
3

In [ ]:
>>> lRDD = sc.parallelize(l)
>>> lRDD.count()
3  

In [ ]:
>>> help(lRDD.flatMap)

>>> s = l[0]
>>> s
'Hello how are you?'
>>> s.split()
['Hello', 'how', 'are', 'you?']
>>> s.split(" ")
['Hello', 'how', 'are', 'you?']
>>> s
'Hello how are you?'

In [ ]:
>>> lFlatMap = lRDD.flatMap(lambda s: s.split(" "))
>>> lFlatMap.count()
10                                                                              
>>> for i in lFlatMap.collect(): print(i)
... 
Hello
how
are
you?
you
are
welcome
welcome
to
xxxxxx

In [ ]:
>>> lMap = lFlatMap.map(lambda s: (s, 1))
>>> lMap.count()
10                                                                              
>>> for i in lMap.collect(): print(i)
... 
('Hello', 1)                                                                    
('how', 1)
('are', 1)
('you?', 1)
('you', 1)
('are', 1)
('welcome', 1)
('welcome', 1)
('to', 1)
('xxxxxx', 1)

In [ ]:
>>> lMap.reduceByKey
<bound method PipelinedRDD.reduceByKey of PythonRDD[7] at collect at <stdin>:1>
>>> wc = lMap.reduceByKey(lambda x, y: x + y)
>>> wc.count()
8

In [ ]:
>>> for i in wc.collect(): print(i)
... 
('you?', 1)
('you', 1)
('xxxxxx', 1)
('to', 1)
('Hello', 1)
('welcome', 2)
('are', 2)
('how', 1)

<p style="background :#d0d5db"><b>Solution to Word Count problem using transformations and action - reduceByKey</b></p>

In [ ]:
>>> lines = sc.textFile("/public/randomtextwriter/part-m-00000")

In [ ]:
>>> type(lines)
<class 'pyspark.rdd.RDD'>

In [ ]:
>>> lines.count()
[Stage 9:>                                                          (0 + 1) / 9]
26421                                                                           
>>> 

In [ ]:
>>> words = lines.flatMap(lambda s: s.split(" "))

In [ ]:
>>> wordTuples = words.map(lambda w: (w, 1))

In [ ]:
>>> for t in wordTuples.take(10): print(t)
... 
(u'SEQ\x06\x19org.apache.hadoop.io.Text\x19org.apache.hadoop.io.Text\x00\x00\x00\x00\x00\x00\ufffdg\x05\x081\ufffd\ufffdJ$\ufffd\u05d2\x1ad\ufffd\x08\x00\x00\x02\ufffd\x00\x00\x00XWpterostigma', 1)
(u'steprelationship', 1)
(u'pleasurehood', 1)
(u'abusiveness', 1)
(u'seelful', 1)
(u'unstipulated', 1)
(u'winterproof', 1)
(u'\ufffd\x02gmericarp', 1)
(u'pentosuria', 1)
(u'airfreighter', 1)

In [ ]:
>>> from operator import add
>>> wordCount = wordTuples.reduceByKey(add)
>>> wordCount.count()
1588631

<p style="background :#d0d5db"><b>Orders -- just example</b></p>

In [ ]:
>>> orders = sc.textFile("/public/retail_db/orders")

>>> orders.first()
u'1,2013-07-25 00:00:00.0,11599,CLOSED'  

In [ ]:
>>> for i in orders.take(10): print(i)
... 
1,2013-07-25 00:00:00.0,11599,CLOSED
2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
3,2013-07-25 00:00:00.0,12111,COMPLETE
4,2013-07-25 00:00:00.0,8827,CLOSED
5,2013-07-25 00:00:00.0,11318,COMPLETE
6,2013-07-25 00:00:00.0,7130,COMPLETE
7,2013-07-25 00:00:00.0,4530,COMPLETE
8,2013-07-25 00:00:00.0,2911,PROCESSING
9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT
10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT

In [ ]:
>>> o = "1,2013-07-25 00:00:00.0,11599,CLOSED"
>>> o.split(",")
['1', '2013-07-25 00:00:00.0', '11599', 'CLOSED']
>>> o.split(",")[3]
'CLOSED'

In [ ]:
>>> orders.map(lambda o: o.split(",")[3]).take(10)
[u'CLOSED', u'PENDING_PAYMENT', u'COMPLETE', u'CLOSED', u'COMPLETE', u'COMPLETE', u'COMPLETE', u'PROCESSING', u'PENDING_PAYMENT', u'PENDING_PAYMENT']

In [ ]:
>>> orders.map(lambda o: int(o.split(",")[1][:4])).take(10)
[2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013]

### <div class="alert alert-success" style="background:#2C3E50;color:white">Basic Transformations and Actions - Shuffling (reduceByKey, groupByKey, aggregateByKey) </div>

<p style="background :#d0d5db"><b>groupByKey & reduceByKey </b></p>

<p style="background :#AED6F1"><b>eg 1 - Compute Revenue for each order id </b></p>

To convert 
**(2, 199.99), (2, 250.0), (2, 129.99) -> [(2, [199.99, 250.0, 129.99])]**
that is input RDD of paired tuples to output RDD which has 1 record having a tuple whose first element is a unique key and the second element is a collection.<br>
**why we need to do this ?**
=> either to add the second elements' of the collection or to sort them or to rank them.
groupByKey facilitates to group the data like this and after that necessary logic can be applied.

In [ ]:
>>> orderItems = sc.textFile("/public/retail_db/order_items/part-00000")
>>> type(orderItems)
<class 'pyspark.rdd.RDD'>

In [ ]:
>>> orderItems.take(10)
[u'1,1,957,1,299.98,299.98', u'2,2,1073,1,199.99,199.99', u'3,2,502,5,250.0,50.0', u'4,2,403,1,129.99,129.99', u'5,4,897,2,49.98,24.99', u'6,4,365,5,299.95,59.99', u'7,4,502,3,150.0,50.0', u'8,4,1014,4,199.92,49.98', u'9,5,957,1,299.98,299.98', u'10,5,365,5,299.95,59.99']

In [ ]:
# for better readability

>>> for oi in orderItems.take(10): print(oi)
... 
1,1,957,1,299.98,299.98
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99
6,4,365,5,299.95,59.99
7,4,502,3,150.0,50.0
8,4,1014,4,199.92,49.98
9,5,957,1,299.98,299.98
10,5,365,5,299.95,59.99

In [ ]:
# just for visualizing what we need as input RDD 
# A tuple with order_id and it's subtotal

>>> oi = "2,2,1073,1,199.99,199.99"
>>> oi.split(",")[1]
'2'
>>> (int(oi.split(",")[1]),float(oi.split(",")[4]))
(2, 199.99)

In [ ]:
# to create a RDD of mapped paired tuples of order_id and it's subtotal

>>> orderItemsMap = orderItems.map(lambda oi: (int(oi.split(",")[1]),float(oi.split(",")[4])))

# just for viewing what the above mapped RDD holds

>>> for i in orderItemsMap.take(10): print(i)
... 
(1, 299.98)                                                                     
(2, 199.99)
(2, 250.0)
(2, 129.99)
(4, 49.98)
(4, 299.95)
(4, 150.0)
(4, 199.92)
(5, 299.98)
(5, 299.95)

In [ ]:
# same as orderItems count, that is total number of records in the data file

>>> orderItemsMap.count()
172198 

<p style="background :#d0d5db"><b>using groupByKey()</b></p>

In [ ]:
# wrongly performed groupByKey on initial input RDD 
# should be performed on orderItemsMap instead

>>> orderItemsMapGBK = orderItems.groupByKey()

In [ ]:
# wrongly counted number of records in it

>>> orderItemsMapGBK.count()
# gives error
# ValueError: too many values to unpack

In [ ]:
# correct groupByKey applied on RDD of mapped paired tuples

>>> orderItemsMapGBK = orderItemsMap.groupByKey()
>>> orderItemsMapGBK.count()
57431

In [ ]:
>>> for i in orderItemsMapGBK: print(i)
... 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
TypeError: 'PipelinedRDD' object is not iterable

In [ ]:
# viewing what grouped by key RDD holds
# (order_id, iterable object <collection>)

>>> for i in orderItemsMapGBK.take(10): print(i)
... 
(2, <pyspark.resultiterable.ResultIterable object at 0x7fcd819f5250>)
(4, <pyspark.resultiterable.ResultIterable object at 0x7fcd819f5410>)
(8, <pyspark.resultiterable.ResultIterable object at 0x7fcd819f5450>)
(10, <pyspark.resultiterable.ResultIterable object at 0x7fcd819f5490>)
(12, <pyspark.resultiterable.ResultIterable object at 0x7fcd819f54d0>)
(14, <pyspark.resultiterable.ResultIterable object at 0x7fcd819f5510>)
(16, <pyspark.resultiterable.ResultIterable object at 0x7fcd819f5550>)
(18, <pyspark.resultiterable.ResultIterable object at 0x7fcd819f5590>)
(20, <pyspark.resultiterable.ResultIterable object at 0x7fcd819f55d0>)
(24, <pyspark.resultiterable.ResultIterable object at 0x7fcd819f5610>)

In [ ]:
# when we use groupByKey for aggregating we again need 
# to create a map for performing aggregation function
# on the grouped by key RDD

>>> orderRevenue = orderItemsMapGBK.map(lambda o: (o[0], sum(o[1])))

In [ ]:
# after above mapping we get the same number of records 
# as in grouped by key RDD, that is what map function
# does -> transforms but returns same number of records

>>> orderRevenue.count()
57431

In [ ]:
# viewing what ouput RDD holds

>>> for rev in orderRevenue.take(10): print(rev)
... 
(2, 579.98)                                                                     
(4, 699.85)
(8, 729.8399999999999)
(10, 651.9200000000001)
(12, 1299.8700000000001)
(14, 549.94)
(16, 419.93)
(18, 449.96000000000004)
(20, 879.8599999999999)
(24, 829.97)

<p style="background :#d0d5db"><b>using reduceByKey()</b></p>

In [ ]:
# performing the aggregation by reduceByKey significantly
# improves the throughput as it doesn't require us to 
# map for the second time and also does shuffling internally

# hence for such scenarios if the problem is to be 
# solved using core APIs of transformations and actions
#reduceByKey should be preferred

>>> orderRevenueRBK = orderItemsMap.reduceByKey(lambda x, y: x + y)
>>> for i in orderRevenueRBK.take(10): print(i)
... 
(2, 579.98)                                                                     
(4, 699.85)
(8, 729.8399999999999)
(10, 651.9200000000001)
(12, 1299.8700000000001)
(14, 549.94)
(16, 419.93)
(18, 449.96000000000004)
(20, 879.8599999999999)
(24, 829.97)

<p style="background:#F1C40F"><b>NOTE :</b> Shuffling includes partitioning, grouping, optionally computing intermediate values.</p>

<p style="background:#F1C40F">Since dynamicAllocation is disabled and num executors is set to 2, the file would have been processed by 2 executors hence in 2 tasks but we wanted to have 4 partitions hence we manually repartitioned it into 4 partitions by using below command.</p><br>

In [ ]:
>>> orderItems = sc.textFile("/public/retail_db/order_items")

In [ ]:
>>> orderItems.repartition(4).saveAsTextFile("/user/monahadoop/pyspark/orderItemsPartitioned")

In [ ]:
[monahadoop@gw03 ~]$ hdfs dfs -ls /user/monahadoop/pyspark/orderItemsPartitioned
Found 5 items
-rw-r--r--   2 monahadoop hdfs          0 2020-06-14 22:21 /user/monahadoop/pyspark/orderItemsPartitioned/_SUCCESS
-rw-r--r--   2 monahadoop hdfs    1351889 2020-06-14 22:21 /user/monahadoop/pyspark/orderItemsPartitioned/part-00000
-rw-r--r--   2 monahadoop hdfs    1352498 2020-06-14 22:21 /user/monahadoop/pyspark/orderItemsPartitioned/part-00001
-rw-r--r--   2 monahadoop hdfs    1351882 2020-06-14 22:21 /user/monahadoop/pyspark/orderItemsPartitioned/part-00002
-rw-r--r--   2 monahadoop hdfs    1352611 2020-06-14 22:21 /user/monahadoop/pyspark/orderItemsPartitioned/part-00003

<p style="background:#F1C40F">To further see that we are indeed using 4 partitions-></p>

In [ ]:
>>> orderItems = sc.textFile("/user/monahadoop/pyspark/orderItemsPartitioned")
>>> orderItems.count()
172198

<p style="background:#F1C40F">By default the log level is other, to see logs and verify that we are now using 4 partitions we can set log level to INFO.</p>

In [ ]:
>>> sc.setLogLevel("INFO")

In [ ]:
>>> orderItems.count()

In [ ]:
>>> orderItemsMap = orderItems.map(lambda oi: (int(oi.split(",")[1]), float(oi.split(",")[4])))

In [ ]:
>>> orderItemsGBK = orderItemsMap.groupByKey(3)
# 3 is optinal argument of number of partitions

### <div class="alert alert-success" style="background:#2C3E50;color:white">Basic Transformations and Actions - filter, joins and sortByKey </div>

<p style="background:#F1C40F"><b>NOTE :</b> As part of lambda functions which are passed to APIs such as map, flatMap, filter etc. - the logic should be pure python.</p>

<p style="background:#AED6F1"><b>eg 1 - Compute Daily Revenue by using filter, joins and sortByKey </b></p>

In [ ]:
>>> orders = sc.textFile("/public/retail_db/orders")

>>> type(orders)
<class 'pyspark.rdd.RDD'>

>>> orders.take(10)
['1,2013-07-25 00:00:00.0,11599,CLOSED', '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT', '3,2013-07-25 00:00:00.0,12111,COMPLETE', '4,2013-07-25 00:00:00.0,8827,CLOSED', '5,2013-07-25 00:00:00.0,11318,COMPLETE', '6,2013-07-25 00:00:00.0,7130,COMPLETE', '7,2013-07-25 00:00:00.0,4530,COMPLETE', '8,2013-07-25 00:00:00.0,2911,PROCESSING', '9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT', '10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT']

In [ ]:
>>> for o in orders.take(10): print(o)
... 
1,2013-07-25 00:00:00.0,11599,CLOSED
2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
3,2013-07-25 00:00:00.0,12111,COMPLETE
4,2013-07-25 00:00:00.0,8827,CLOSED
5,2013-07-25 00:00:00.0,11318,COMPLETE
6,2013-07-25 00:00:00.0,7130,COMPLETE
7,2013-07-25 00:00:00.0,4530,COMPLETE
8,2013-07-25 00:00:00.0,2911,PROCESSING
9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT
10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT

In [ ]:
>>> ordersFiltered = orders.filter(lambda o: o.split(",")[3] in('CLOSED', 'COMPLETE'))

In [ ]:
>>> type(ordersFiltered)
<class 'pyspark.rdd.PipelinedRDD'>

In [ ]:
>>> ordersFiltered.count()
30455  

In [ ]:
>>> orderItems = sc.textFile("/public/retail_db/order_items")

In [ ]:
>>> for oi in orderItems.takeSample(True, 10): print(oi)
... 
62021,24759,191,1,99.99,99.99                                                   
49920,19967,403,1,129.99,129.99
82937,33177,365,3,179.97,59.99
63944,25522,823,1,51.99,51.99
101697,40757,502,5,250.0,50.0
110762,44368,403,1,129.99,129.99
12278,4910,957,1,299.98,299.98
40742,16321,502,5,250.0,50.0
72244,28869,191,2,199.98,99.99
116843,46732,957,1,299.98,299.98

In [ ]:
>>> ordersMap = ordersFiltered.map(lambda o: (int(o.split(",")[0]), o.split(",")[1]))

In [ ]:
>>> for i in ordersMap.take(10): print(i)
... 
(1, '2013-07-25 00:00:00.0')
(3, '2013-07-25 00:00:00.0')
(4, '2013-07-25 00:00:00.0')
(5, '2013-07-25 00:00:00.0')
(6, '2013-07-25 00:00:00.0')
(7, '2013-07-25 00:00:00.0')
(12, '2013-07-25 00:00:00.0')
(15, '2013-07-25 00:00:00.0')
(17, '2013-07-25 00:00:00.0')
(18, '2013-07-25 00:00:00.0')

In [ ]:
>>> orderItemsMap = orderItems.map(lambda oi: (int(oi.split(",")[1]), float(oi.split(",")[4])))

In [ ]:
>>> for i in orderItemsMap.take(10): print(i)
... 
(1, 299.98)                                                                     
(2, 199.99)
(2, 250.0)
(2, 129.99)
(4, 49.98)
(4, 299.95)
(4, 150.0)
(4, 199.92)
(5, 299.98)
(5, 299.95)

In [ ]:
>>> ordersJoin = ordersMap.join(orderItemsMap)

In [ ]:
>>> for i in ordersJoin.take(10): print(i)
... 
(4, ('2013-07-25 00:00:00.0', 49.98))                                           
(4, ('2013-07-25 00:00:00.0', 299.95))
(4, ('2013-07-25 00:00:00.0', 150.0))
(4, ('2013-07-25 00:00:00.0', 199.92))
(12, ('2013-07-25 00:00:00.0', 299.98))
(12, ('2013-07-25 00:00:00.0', 100.0))
(12, ('2013-07-25 00:00:00.0', 149.94))
(12, ('2013-07-25 00:00:00.0', 499.95))
(12, ('2013-07-25 00:00:00.0', 250.0))
(24, ('2013-07-25 00:00:00.0', 129.99))

In [ ]:
>>> for i in ordersJoin.take(10): print(i)
... 
(35228, ('2014-02-27 00:00:00.0', 249.9))
(35228, ('2014-02-27 00:00:00.0', 399.98))
(35232, ('2014-02-27 00:00:00.0', 100.0))
(35248, ('2014-02-27 00:00:00.0', 199.92))
(35264, ('2014-02-27 00:00:00.0', 200.0))
(35264, ('2014-02-27 00:00:00.0', 239.96))
(35272, ('2014-02-27 00:00:00.0', 30.0))
(35272, ('2014-02-27 00:00:00.0', 129.99))
(35272, ('2014-02-27 00:00:00.0', 59.99))
(35272, ('2014-02-27 00:00:00.0', 299.98))

In [ ]:
>>> ordersJoinMap = ordersJoin.map(lambda o: o[1])

In [ ]:
>>> for i in ordersJoinMap.take(10): print(i)
... 
('2013-07-25 00:00:00.0', 49.98)                                                
('2013-07-25 00:00:00.0', 299.95)
('2013-07-25 00:00:00.0', 150.0)
('2013-07-25 00:00:00.0', 199.92)
('2013-07-25 00:00:00.0', 299.98)
('2013-07-25 00:00:00.0', 100.0)
('2013-07-25 00:00:00.0', 149.94)
('2013-07-25 00:00:00.0', 499.95)
('2013-07-25 00:00:00.0', 250.0)
('2013-07-25 00:00:00.0', 129.99)

In [ ]:
>>> ordersJoinMap.count()
75408

In [ ]:
>>> DailyRevenue = ordersJoinMap.reduceByKey(lambda x,y: x + y)

In [ ]:
#other way to add

>>> from operator import add
>>> DailyRevenue = ordersJoinMap.reduceByKey(add)

In [ ]:
>>> DailyRevenue.count()
[Stage 39:===========================================>              (3 + 1) / 4]
364

In [ ]:
>>> for i in DailyRevenue.take(10): print(i)
... 
('2014-03-03 00:00:00.0', 52553.409999999974)
('2014-03-05 00:00:00.0', 43432.309999999976)
('2014-03-06 00:00:00.0', 42483.269999999975)
('2014-03-07 00:00:00.0', 37843.39999999998)
('2014-03-12 00:00:00.0', 54095.61999999995)
('2014-03-18 00:00:00.0', 45921.39999999997)
('2014-03-25 00:00:00.0', 27971.80999999999)
('2014-03-26 00:00:00.0', 57003.34999999995)
('2014-03-27 00:00:00.0', 28715.84999999999)
('2014-04-04 00:00:00.0', 44999.059999999976)

In [ ]:
>>> DailyRevenue = ordersJoinMap.reduceByKey(lambda x,y: round((x + y), 2))

In [ ]:
>>> for i in DailyRevenue.take(10): print(i)
... 
('2014-03-03 00:00:00.0', 52553.41)                                             
('2014-03-05 00:00:00.0', 43432.31)
('2014-03-06 00:00:00.0', 42483.27)
('2014-03-07 00:00:00.0', 37843.4)
('2014-03-12 00:00:00.0', 54095.62)
('2014-03-18 00:00:00.0', 45921.4)
('2014-03-25 00:00:00.0', 27971.81)
('2014-03-26 00:00:00.0', 57003.35)
('2014-03-27 00:00:00.0', 28715.85)
('2014-04-04 00:00:00.0', 44999.06)

In [ ]:
#sorting daily_revenue in asc order of date

>>> DailyRevenueSorted = DailyRevenue.sortByKey()

In [ ]:
>>> for i in DailyRevenueSorted.take(10): print(i)
... 
('2013-07-25 00:00:00.0', 31547.23)
('2013-07-26 00:00:00.0', 54713.23)
('2013-07-27 00:00:00.0', 48411.48)
('2013-07-28 00:00:00.0', 35672.03)
('2013-07-29 00:00:00.0', 54579.7)
('2013-07-30 00:00:00.0', 49329.29)
('2013-07-31 00:00:00.0', 59212.49)
('2013-08-01 00:00:00.0', 49160.08)
('2013-08-02 00:00:00.0', 50688.58)
('2013-08-03 00:00:00.0', 43416.74)

In [ ]:
#sorting daily_revenue in desc order of date

>>> DailyRevenueSorted = DailyRevenue.sortByKey(False)

In [ ]:
>>> for i in DailyRevenueSorted.take(10): print(i)
... 
('2014-07-24 00:00:00.0', 50885.19)
('2014-07-23 00:00:00.0', 38795.23)
('2014-07-22 00:00:00.0', 36717.24)
('2014-07-21 00:00:00.0', 51427.7)
('2014-07-20 00:00:00.0', 60047.45)
('2014-07-19 00:00:00.0', 38420.99)
('2014-07-18 00:00:00.0', 43856.6)
('2014-07-17 00:00:00.0', 36384.77)
('2014-07-16 00:00:00.0', 43011.92)
('2014-07-15 00:00:00.0', 53480.23)

In [ ]:
>>> DailyRevenueSorted = DailyRevenue.sortByKey(True)

>>> for i in DailyRevenueSorted.take(10): print(i)                              
... 
('2013-07-25 00:00:00.0', 31547.23)
('2013-07-26 00:00:00.0', 54713.23)
('2013-07-27 00:00:00.0', 48411.48)
('2013-07-28 00:00:00.0', 35672.03)
('2013-07-29 00:00:00.0', 54579.7)
('2013-07-30 00:00:00.0', 49329.29)
('2013-07-31 00:00:00.0', 59212.49)
('2013-08-01 00:00:00.0', 49160.08)
('2013-08-02 00:00:00.0', 50688.58)
('2013-08-03 00:00:00.0', 43416.74)

<p style="background:#F1C40F"><b>NOTE :</b> Transforming into string format in which this data is to be saved into file. Comma Separated Values instead of tuples.<br><code>lambda o: o[0] + "," + str(o[1])</code></p>

In [ ]:
>>> # transforming into string format in which this data is to be saved into file

>>> for i in DailyRevenueSorted.map(lambda o: o[0] + "," + str(o[1])).take(10):
...     print(i)
... 
2013-07-25 00:00:00.0,31547.23                                                  
2013-07-26 00:00:00.0,54713.23
2013-07-27 00:00:00.0,48411.48
2013-07-28 00:00:00.0,35672.03
2013-07-29 00:00:00.0,54579.7
2013-07-30 00:00:00.0,49329.29
2013-07-31 00:00:00.0,59212.49
2013-08-01 00:00:00.0,49160.08
2013-08-02 00:00:00.0,50688.58
2013-08-03 00:00:00.0,43416.74

In [ ]:
>>> DailyRevenueSorted.map(lambda o: o[0] + "," + str(o[1])).saveAsTextFile("/user/monahadoop/pyspark/daily_revenue")

In [ ]:
[monahadoop@gw03 ~]$ hdfs dfs -ls /user/monahadoop/pyspark/daily_revenue
Found 5 items
-rw-r--r--   2 monahadoop hdfs          0 2020-06-16 03:54 /user/monahadoop/pyspark/daily_revenue/_SUCCESS
-rw-r--r--   2 monahadoop hdfs       3086 2020-06-16 03:54 /user/monahadoop/pyspark/daily_revenue/part-00000
-rw-r--r--   2 monahadoop hdfs       1667 2020-06-16 03:54 /user/monahadoop/pyspark/daily_revenue/part-00001
-rw-r--r--   2 monahadoop hdfs       3921 2020-06-16 03:54 /user/monahadoop/pyspark/daily_revenue/part-00002
-rw-r--r--   2 monahadoop hdfs       2565 2020-06-16 03:54 /user/monahadoop/pyspark/daily_revenue/part-00003

[monahadoop@gw03 ~]$ hdfs dfs -cat /user/monahadoop/pyspark/daily_revenue/part-00003
2014-05-03 00:00:00.0,49541.22
2014-05-04 00:00:00.0,31853.8
2014-05-05 00:00:00.0,37446.95
2014-05-06 00:00:00.0,60351.47
2014-05-07 00:00:00.0,45749.36
2014-05-08 00:00:00.0,23799.37

<div><br></div>

<p style="background:#AED6F1"><b>eg2  - left and right outer joins </b></p>

<p style="background:#F1C40F"><b>NOTE :</b> To apply join between to RDDs we need to first convert them into (K,V) pairs or paired tuples.</p>

<p style="background:#F1C40F"><b>NOTE :</b> To convert orders and orderItems RDDs to paired tuples -> lambda function would contain (order_id, whole RDD) as below:<br><code>lambda o: (int(o.split(",")[0]), o)</code></p>

In [ ]:
>>> #joining orders and order_items whole

In [ ]:
>>> ordersMap1 = orders.map(lambda o: (int(o.split(",")[0]), o))

>>> for i in ordersMap1.take(10): print(i)
... 
(1, '1,2013-07-25 00:00:00.0,11599,CLOSED')                                     
(2, '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT')
(3, '3,2013-07-25 00:00:00.0,12111,COMPLETE')
(4, '4,2013-07-25 00:00:00.0,8827,CLOSED')
(5, '5,2013-07-25 00:00:00.0,11318,COMPLETE')
(6, '6,2013-07-25 00:00:00.0,7130,COMPLETE')
(7, '7,2013-07-25 00:00:00.0,4530,COMPLETE')
(8, '8,2013-07-25 00:00:00.0,2911,PROCESSING')
(9, '9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT')
(10, '10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT')


In [ ]:
>>> orderItemsMap1 = orderItems.map(lambda oi: (int(oi.split(",")[1]), oi))

>>> for i in orderItemsMap1.take(10): print(i)
... 
(1, '1,1,957,1,299.98,299.98')                                                  
(2, '2,2,1073,1,199.99,199.99')
(2, '3,2,502,5,250.0,50.0')
(2, '4,2,403,1,129.99,129.99')
(4, '5,4,897,2,49.98,24.99')
(4, '6,4,365,5,299.95,59.99')
(4, '7,4,502,3,150.0,50.0')
(4, '8,4,1014,4,199.92,49.98')
(5, '9,5,957,1,299.98,299.98')
(5, '10,5,365,5,299.95,59.99')


In [ ]:
#for extracting distinct records

>>> ordersMap1.join(orderItemsMap1).map(lambda o: o[0]).distinct().count()
57431

<p style="background :#d0d5db"><b>Left Outer Join</b></p>

In [ ]:
# left outer join to find orders with no order_items - from business point of view it cannot and should not be
# all data from left table and matching + Null from right table

>>> ordersLOJoind = ordersMap1.leftOuterJoin(orderItemsMap1)

>>> for i in ordersLOJoind.take(10): print(i)
... 
(4, ('4,2013-07-25 00:00:00.0,8827,CLOSED', '5,4,897,2,49.98,24.99'))           
(4, ('4,2013-07-25 00:00:00.0,8827,CLOSED', '6,4,365,5,299.95,59.99'))
(4, ('4,2013-07-25 00:00:00.0,8827,CLOSED', '7,4,502,3,150.0,50.0'))
(4, ('4,2013-07-25 00:00:00.0,8827,CLOSED', '8,4,1014,4,199.92,49.98'))
(8, ('8,2013-07-25 00:00:00.0,2911,PROCESSING', '17,8,365,3,179.97,59.99'))
(8, ('8,2013-07-25 00:00:00.0,2911,PROCESSING', '18,8,365,5,299.95,59.99'))
(8, ('8,2013-07-25 00:00:00.0,2911,PROCESSING', '19,8,1014,4,199.92,49.98'))
(8, ('8,2013-07-25 00:00:00.0,2911,PROCESSING', '20,8,502,1,50.0,50.0'))
(12, ('12,2013-07-25 00:00:00.0,1837,CLOSED', '34,12,957,1,299.98,299.98'))
(12, ('12,2013-07-25 00:00:00.0,1837,CLOSED', '35,12,134,4,100.0,25.0'))

In [ ]:
# ordersWithNoOrderItems contains orders which have Null in order items

>>> ordersWithNoOrderItems = ordersLOJoind.filter(lambda o: o[1][1] == None)

>>> ordersWithNoOrderItems.count()
11452

In [ ]:
>>> for i in ordersWithNoOrderItems.take(100): print(i)
... 
(34568, ('34568,2014-02-23 00:00:00.0,1271,COMPLETE', None))
(34572, ('34572,2014-02-23 00:00:00.0,8135,PENDING', None))
(34580, ('34580,2014-02-23 00:00:00.0,6540,COMPLETE', None))
(34688, ('34688,2014-02-24 00:00:00.0,8033,SUSPECTED_FRAUD', None))
(34704, ('34704,2014-02-24 00:00:00.0,2858,COMPLETE', None))
(34812, ('34812,2014-02-24 00:00:00.0,8435,COMPLETE', None))
(34872, ('34872,2014-02-25 00:00:00.0,9176,PENDING_PAYMENT', None))
(34896, ('34896,2014-02-25 00:00:00.0,10749,PENDING', None))
(34920, ('34920,2014-02-25 00:00:00.0,10997,COMPLETE', None)) 

In [ ]:
# EXAMPLE -> SAMPLE DATA TO CREATE LOGIC

>>> o = (36888, ('36888,2014-03-08 00:00:00.0,5248,PENDING', None))
>>> type(o)
<class 'tuple'>
>>> o[0]
36888
>>> o[1]
('36888,2014-03-08 00:00:00.0,5248,PENDING', None)
>>> o[1][0]
'36888,2014-03-08 00:00:00.0,5248,PENDING'
>>> o[1][1]
>>> o[1][1] == None
True

<p style="background :#d0d5db"><b>Right Outer Join</b></p>

In [ ]:
>>> ordersROJoin = orderItemsMap1.rightOuterJoin(ordersMap1)

In [ ]:
>>> for i in ordersROJoin.take(100): print(i)
... 
(4, ('5,4,897,2,49.98,24.99', '4,2013-07-25 00:00:00.0,8827,CLOSED'))           
(4, ('6,4,365,5,299.95,59.99', '4,2013-07-25 00:00:00.0,8827,CLOSED'))
(4, ('7,4,502,3,150.0,50.0', '4,2013-07-25 00:00:00.0,8827,CLOSED'))
(4, ('8,4,1014,4,199.92,49.98', '4,2013-07-25 00:00:00.0,8827,CLOSED'))
(8, ('17,8,365,3,179.97,59.99', '8,2013-07-25 00:00:00.0,2911,PROCESSING'))
(8, ('18,8,365,5,299.95,59.99', '8,2013-07-25 00:00:00.0,2911,PROCESSING'))
(8, ('19,8,1014,4,199.92,49.98', '8,2013-07-25 00:00:00.0,2911,PROCESSING'))
(8, ('20,8,502,1,50.0,50.0', '8,2013-07-25 00:00:00.0,2911,PROCESSING'))
(12, ('34,12,957,1,299.98,299.98', '12,2013-07-25 00:00:00.0,1837,CLOSED'))

In [ ]:
>>> ordersROJoin.count()
183650
>>> ordersLOJoind.count()
183650

In [ ]:
>>> ordersWithNoOrderItems = ordersROJoin.filter(lambda o: (o[1][0] == None))

>>> ordersWithNoOrderItems.count()
11452

In [ ]:
>>> for i in ordersWithNoOrderItems.take(100): print(i)
... 
(34568, (None, '34568,2014-02-23 00:00:00.0,1271,COMPLETE'))
(34572, (None, '34572,2014-02-23 00:00:00.0,8135,PENDING'))
(34580, (None, '34580,2014-02-23 00:00:00.0,6540,COMPLETE'))
(34688, (None, '34688,2014-02-24 00:00:00.0,8033,SUSPECTED_FRAUD'))
(34704, (None, '34704,2014-02-24 00:00:00.0,2858,COMPLETE'))

<div><br></div>

<p style="background:#AED6F1"><b>eg3  - composite sorting </b></p>

<p style="background:#F1C40F"><b>NOTE : </b>Composite sorting means sorting data based on two keys' values. In the example below, data is sorted based on keys - order_id and subtotal. We need only 1 RDD i.e. orderItems.</p>

In [ ]:
>>> for i in orderItems.take(10): print(i)
... 
1,1,957,1,299.98,299.98                                                         
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99
6,4,365,5,299.95,59.99
7,4,502,3,150.0,50.0
8,4,1014,4,199.92,49.98
9,5,957,1,299.98,299.98
10,5,365,5,299.95,59.99

<p style="background:#F1C40F"><b>step 1 : </b>Below creating a map of order_items, which results in nested paired tuple for each record.<br>As in, ((oi.order_id, oi. subtotal), orderItems)<br><code>lambda oi: ((int(oi.split(",")[1]), float(oi.split(",")[4])), oi)</code></p>

In [ ]:
>>> oiMap = orderItems.map(lambda oi: ((int(oi.split(",")[1]), float(oi.split(",")[4])), oi))

In [ ]:
>>> for i in oiMap.take(10): print(i)
... 
((1, 299.98), '1,1,957,1,299.98,299.98')                                        
((2, 199.99), '2,2,1073,1,199.99,199.99')
((2, 250.0), '3,2,502,5,250.0,50.0')
((2, 129.99), '4,2,403,1,129.99,129.99')
((4, 49.98), '5,4,897,2,49.98,24.99')
((4, 299.95), '6,4,365,5,299.95,59.99')
((4, 150.0), '7,4,502,3,150.0,50.0')
((4, 199.92), '8,4,1014,4,199.92,49.98')
((5, 299.98), '9,5,957,1,299.98,299.98')
((5, 299.95), '10,5,365,5,299.95,59.99')

<p style="background:#F1C40F"><b>step 2 : </b>Sort above data in ascending order of key. Key being (oi.order_id, oi. subtotal). sortByKey() has True as default argument. True for sorting in ascending order.<br><code>(int(oi.split(",")[1]), float(oi.split(",")[4])) -> sortByKey()</code></p>

In [ ]:
>>> for i in oiMap.sortByKey().take(10): print(i)
... 
((1, 299.98), '1,1,957,1,299.98,299.98')                                        
((2, 129.99), '4,2,403,1,129.99,129.99')
((2, 199.99), '2,2,1073,1,199.99,199.99')
((2, 250.0), '3,2,502,5,250.0,50.0')
((4, 49.98), '5,4,897,2,49.98,24.99')
((4, 150.0), '7,4,502,3,150.0,50.0')
((4, 199.92), '8,4,1014,4,199.92,49.98')
((4, 299.95), '6,4,365,5,299.95,59.99')
((5, 99.96), '11,5,1014,2,99.96,49.98')
((5, 129.99), '13,5,403,1,129.99,129.99')

<p style="background:#F1C40F"><b>step 3 : </b>Sort above data in descending order of key. Key being (oi.order_id, oi. subtotal). sortByKey(False). False for sorting in descending order.<br><code>(int(oi.split(",")[1]), float(oi.split(",")[4])) -> sortByKey(False)</code></p>

In [ ]:
>>> for i in oiMap.sortByKey(False).take(10): print(i)
... 
((68883, 1999.99), '172197,68883,208,1,1999.99,1999.99')                        
((68883, 150.0), '172198,68883,502,3,150.0,50.0')
((68882, 59.99), '172195,68882,365,1,59.99,59.99')
((68882, 50.0), '172196,68882,502,1,50.0,50.0')
((68881, 129.99), '172194,68881,403,1,129.99,129.99')
((68880, 250.0), '172190,68880,502,5,250.0,50.0')
((68880, 249.9), '172192,68880,1014,5,249.9,49.98')
((68880, 199.99), '172191,68880,1073,1,199.99,199.99')
((68880, 149.94), '172189,68880,1014,3,149.94,49.98')
((68880, 149.94), '172193,68880,1014,3,149.94,49.98')

<p style="background:#F1C40F"><b>step 4 : </b> Now if we want to sort records in ascending order of order_id first and descending order of subtotal within, then using a trick <br>-> making subtotals as -ve, will arrange them in descending order of their values (but ascending order wrt -ve value) <br>-> hence the corresponding records will be arranged in order <br>-> order_id asc, subtotal desc.<br>Later we could discard the keys used for sorting and just display sorted records as output. <br><code>(int(oi.split(",")[1]), -float(oi.split(",")[4])) -> sortByKey(False)</code></p>

In [ ]:
>>> oiMap = orderItems.map(lambda oi: ((int(oi.split(",")[1]), -float(oi.split(",")[4])), oi))

In [ ]:
>>> for i in oiMap.sortByKey().take(10): print(i)
... 
((1, -299.98), '1,1,957,1,299.98,299.98')                                       
((2, -250.0), '3,2,502,5,250.0,50.0')
((2, -199.99), '2,2,1073,1,199.99,199.99')
((2, -129.99), '4,2,403,1,129.99,129.99')
((4, -299.95), '6,4,365,5,299.95,59.99')
((4, -199.92), '8,4,1014,4,199.92,49.98')
((4, -150.0), '7,4,502,3,150.0,50.0')
((4, -49.98), '5,4,897,2,49.98,24.99')
((5, -299.98), '9,5,957,1,299.98,299.98')
((5, -299.98), '12,5,957,1,299.98,299.98')

In [ ]:
>>> #once the data is sorted based on composite key in this case output above is sorted by 
... #order id ascending and subtotal descending, we can discard the key and display the sorted data 
... # as follows

>>> for i in oiMap.sortByKey(True).map(lambda o: o[1]).take(10): print(i)
... 
1,1,957,1,299.98,299.98                                                         
3,2,502,5,250.0,50.0
2,2,1073,1,199.99,199.99
4,2,403,1,129.99,129.99
6,4,365,5,299.95,59.99
8,4,1014,4,199.92,49.98
7,4,502,3,150.0,50.0
5,4,897,2,49.98,24.99
9,5,957,1,299.98,299.98
12,5,957,1,299.98,299.98


### <div class="alert alert-success" style="background:#2C3E50;color:white">Accumulators and Broadcast Variables, Repartition and Coalesce 01</div>

<p style="background:#F1C40F"><b>Accumulators and Broadcast variables</b></p> 

* are also known as Shared Variables.
* Accumulators are primarily used as counters for sanity checks.
* Broadcast variables are used for lookups.

<p style="background:#AED6F1"><b>Revenue per Product for a given month using Accumulators</p>

<p style="background :#d0d5db"><b>Problem Statement</b> </p>

How can we use HDFS APIs as part of applications built using pyspark.
* We have to use orders, order_items and products data sets to compute revenue per product for a given month.
* From  orders -> order_id and order_date.
* order_items -> order_item_order_id, order_item_subtotal and order_item_product_id.
* products -> product_id, product_name.
* orders and order_items are in HDFS and products is in local file system.
* High level design
    * Accept year and month as program argument (along with input and output paths).
    * Filter for orders which fall in that month(args).
    * Join filtered orders and order_items to get order_item details for that month.
    * Get revenue for each product id.
    * We need to read products from local file system.
    * Convert it into RDD and extract product_id and name.
    * Join it with aggregated order_items.
    * Get product name and revenue for each product.
* Application properties.
    

<p style="background :#d0d5db"><b>xxxx</b> </p>

#(order_id, 1)

#(order_item_order_id, (order_item_product_id, order_item_subtotal))

#After join (order_id, ((order_item_product_id, order_item_subtotal), 1))

(1300, ((191, 199.98), 1))                                                      
(1300, ((1014, 199.92), 1))

#After map over join -> for discarding the order_id and 1 from the joined result as we just need product_id as key and sum up the sub_total values in accordance.
rec[1][0] ->
(order_item_product_id, order_item_subtotal)

#After reduceByKey we get -> order_item_product_id, product_revenue for that month.
#then we need to join product and order_item to get product name and display product_name and product_revenue as output

### <div class="alert alert-success" style="background:#2C3E50;color:white">Creating Data Frames and Pre Defined Functions</div>

#### To work with Data Frames as well as Spark SQL, we need to create object of type Spark Session.

>>>SparkSession available as 'spark'.
>>> spark
<pyspark.sql.session.SparkSession object at 0x7fcba9bf3d68>

>>> sc
<SparkContext master=yarn appName=PySparkShell>
    
>>> orders = sc.textFile("/public/retail_db/orders")
>>> type(orders)
<class 'pyspark.rdd.RDD'>
>>> orders.first()
'1,2013-07-25 00:00:00.0,11599,CLOSED'                                          
>>> type(orders.first())
<class 'str'>

**RDD is structure less

A DF is nothing but an RDD with structure(schema). As it has structure, we should be able to select certain fields using their names.

Once we create DF we can actually read the cols using attributes.
creating schema using toDF() fucntion, using this func we can define names for each cols.

In [ ]:
>>> ordersDF = sc.read.csv('/public/retail_db/orders').toDF('order_id', 'order_date', 'order_cust_id', 'order_status')
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
AttributeError: 'SparkContext' object has no attribute 'read'

#to give names to DF's attributes
>>> ordersDF = spark.read.csv('/public/retail_db/orders').toDF('order_id', 'order_date', 'order_cust_id', 'order_status')
>>> ordersDF.printSchema()
root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_cust_id: string (nullable = true)
 |-- order_status: string (nullable = true)

>>> 

Attributes/cols in a DF can be referred using names.

In [ ]:
>>> ordersDF.select('order_id', 'order_date').show()
+--------+--------------------+                                                 
|order_id|          order_date|
+--------+--------------------+
|       1|2013-07-25 00:00:...|
|       2|2013-07-25 00:00:...|
|       3|2013-07-25 00:00:...|
|       4|2013-07-25 00:00:...|
|       5|2013-07-25 00:00:...|
|       6|2013-07-25 00:00:...|
|       7|2013-07-25 00:00:...|
|       8|2013-07-25 00:00:...|
|       9|2013-07-25 00:00:...|
|      10|2013-07-25 00:00:...|
|      11|2013-07-25 00:00:...|
|      12|2013-07-25 00:00:...|
|      13|2013-07-25 00:00:...|
|      14|2013-07-25 00:00:...|
|      15|2013-07-25 00:00:...|
|      16|2013-07-25 00:00:...|
|      17|2013-07-25 00:00:...|
|      18|2013-07-25 00:00:...|
|      19|2013-07-25 00:00:...|
|      20|2013-07-25 00:00:...|
+--------+--------------------+
only showing top 20 rows

when we use APIs like select on DF most of the APIs return a DF again. To preview the data in the DF, there's a function called show(), if you apply show().. it wiil show those fields upto 20 records by default.

In [ ]:
>>> ordersDF = spark.read.csv('/public/retail_db/orders')
>>> ordersDF.show()                                                             
+---+--------------------+-----+---------------+
|_c0|                 _c1|  _c2|            _c3|
+---+--------------------+-----+---------------+
|  1|2013-07-25 00:00:...|11599|         CLOSED|
|  2|2013-07-25 00:00:...|  256|PENDING_PAYMENT|
|  3|2013-07-25 00:00:...|12111|       COMPLETE|
|  4|2013-07-25 00:00:...| 8827|         CLOSED|
|  5|2013-07-25 00:00:...|11318|       COMPLETE|
|  6|2013-07-25 00:00:...| 7130|       COMPLETE|
|  7|2013-07-25 00:00:...| 4530|       COMPLETE|
|  8|2013-07-25 00:00:...| 2911|     PROCESSING|
|  9|2013-07-25 00:00:...| 5657|PENDING_PAYMENT|
| 10|2013-07-25 00:00:...| 5648|PENDING_PAYMENT|
| 11|2013-07-25 00:00:...|  918| PAYMENT_REVIEW|
| 12|2013-07-25 00:00:...| 1837|         CLOSED|
| 13|2013-07-25 00:00:...| 9149|PENDING_PAYMENT|
| 14|2013-07-25 00:00:...| 9842|     PROCESSING|
| 15|2013-07-25 00:00:...| 2568|       COMPLETE|
| 16|2013-07-25 00:00:...| 7276|PENDING_PAYMENT|
| 17|2013-07-25 00:00:...| 2667|       COMPLETE|
| 18|2013-07-25 00:00:...| 1205|         CLOSED|
| 19|2013-07-25 00:00:...| 9488|PENDING_PAYMENT|
| 20|2013-07-25 00:00:...| 9198|     PROCESSING|
+---+--------------------+-----+---------------+
only showing top 20 rows


In [ ]:
>>> ordersDF.show(5)
+---+--------------------+-----+---------------+
|_c0|                 _c1|  _c2|            _c3|
+---+--------------------+-----+---------------+
|  1|2013-07-25 00:00:...|11599|         CLOSED|
|  2|2013-07-25 00:00:...|  256|PENDING_PAYMENT|
|  3|2013-07-25 00:00:...|12111|       COMPLETE|
|  4|2013-07-25 00:00:...| 8827|         CLOSED|
|  5|2013-07-25 00:00:...|11318|       COMPLETE|
+---+--------------------+-----+---------------+
only showing top 5 rows

In [ ]:
>>> ordersDF.limit(5).show()
+---+--------------------+-----+---------------+
|_c0|                 _c1|  _c2|            _c3|
+---+--------------------+-----+---------------+
|  1|2013-07-25 00:00:...|11599|         CLOSED|
|  2|2013-07-25 00:00:...|  256|PENDING_PAYMENT|
|  3|2013-07-25 00:00:...|12111|       COMPLETE|
|  4|2013-07-25 00:00:...| 8827|         CLOSED|
|  5|2013-07-25 00:00:...|11318|       COMPLETE|
+---+--------------------+-----+---------------+

In [ ]:
# take(20) converts 20 records to an array
>>> ordersDF.select('order_id', 'order_date').take(20)

In [ ]:
[Row(order_id='1', order_date='2013-07-25 00:00:00.0'), Row(order_id='2', order_date='2013-07-25 00:00:00.0'), Row(order_id='3', order_date='2013-07-25 00:00:00.0'), Row(order_id='4', order_date='2013-07-25 00:00:00.0'), Row(order_id='5', order_date='2013-07-25 00:00:00.0'), Row(order_id='6', order_date='2013-07-25 00:00:00.0'), Row(order_id='7', order_date='2013-07-25 00:00:00.0'), Row(order_id='8', order_date='2013-07-25 00:00:00.0'), Row(order_id='9', order_date='2013-07-25 00:00:00.0'), Row(order_id='10', order_date='2013-07-25 00:00:00.0'), Row(order_id='11', order_date='2013-07-25 00:00:00.0'), Row(order_id='12', order_date='2013-07-25 00:00:00.0'), Row(order_id='13', order_date='2013-07-25 00:00:00.0'), Row(order_id='14', order_date='2013-07-25 00:00:00.0'), Row(order_id='15', order_date='2013-07-25 00:00:00.0'), Row(order_id='16', order_date='2013-07-25 00:00:00.0'), Row(order_id='17', order_date='2013-07-25 00:00:00.0'), Row(order_id='18', order_date='2013-07-25 00:00:00.0'), Row(order_id='19', order_date='2013-07-25 00:00:00.0'), Row(order_id='20', order_date='2013-07-25 00:00:00.0')]

In [ ]:
# collect() converts all records into an array
>>> ordersDF.select('order_id', 'order_date').collect()

In [ ]:
# for loop below improves readbility
>>> for i in ordersDF.select('order_id', 'order_date').take(20): print(i)
... 
Row(order_id='1', order_date='2013-07-25 00:00:00.0')                           
Row(order_id='2', order_date='2013-07-25 00:00:00.0')
Row(order_id='3', order_date='2013-07-25 00:00:00.0')
Row(order_id='4', order_date='2013-07-25 00:00:00.0')
Row(order_id='5', order_date='2013-07-25 00:00:00.0')
Row(order_id='6', order_date='2013-07-25 00:00:00.0')
Row(order_id='7', order_date='2013-07-25 00:00:00.0')
Row(order_id='8', order_date='2013-07-25 00:00:00.0')
Row(order_id='9', order_date='2013-07-25 00:00:00.0')
Row(order_id='10', order_date='2013-07-25 00:00:00.0')
Row(order_id='11', order_date='2013-07-25 00:00:00.0')
Row(order_id='12', order_date='2013-07-25 00:00:00.0')
Row(order_id='13', order_date='2013-07-25 00:00:00.0')
Row(order_id='14', order_date='2013-07-25 00:00:00.0')
Row(order_id='15', order_date='2013-07-25 00:00:00.0')
Row(order_id='16', order_date='2013-07-25 00:00:00.0')
Row(order_id='17', order_date='2013-07-25 00:00:00.0')
Row(order_id='18', order_date='2013-07-25 00:00:00.0')
Row(order_id='19', order_date='2013-07-25 00:00:00.0')
Row(order_id='20', order_date='2013-07-25 00:00:00.0')

In [ ]:
# describe will give lot more details of your schema than previewing 
# or priniting the schema

In [ ]:
>>> ordersDF.describe()
DataFrame[summary: string, order_id: string, order_date: string, order_cust_id: string, order_status: string]

In [ ]:
>>> ordersDF.describe().show()
+-------+------------------+--------------------+-----------------+---------------+
|summary|          order_id|          order_date|    order_cust_id|   order_status|
+-------+------------------+--------------------+-----------------+---------------+
|  count|             68883|               68883|            68883|          68883|
|   mean|           34442.0|                null|6216.571098819738|           null|
| stddev|19884.953633337947|                null|3586.205241263963|           null|
|    min|                 1|2013-07-25 00:00:...|                1|       CANCELED|
|    max|              9999|2014-07-24 00:00:...|             9999|SUSPECTED_FRAUD|
order+-------+------------------+--------------------+-----------------+---------------+

In [ ]:
>>> ordersDF.count()
68883

Once DF is created it can be processed using two approaches:
* Native DF APIs - eg select(<fields_name>)
* Register DF as Temp Table and run queries against it using spark.sql

In [ ]:
# createTempView creates a temp table/view named orders 
# on orders we can then use sql API of spark object to run sql queries

>>> ordersDF.createTempView('orders')

In [ ]:
# below command also creates a DF and using show() on it we can view the data.

>>> spark.sql('select * from orders')

DataFrame[order_id: string, order_date: string, order_cust_id: string, order_status: string]

In [ ]:
>>> spark.sql('select * from orders').show()
+--------+--------------------+-------------+---------------+                   
|order_id|          order_date|order_cust_id|   order_status|
+--------+--------------------+-------------+---------------+
|       1|2013-07-25 00:00:...|        11599|         CLOSED|
|       2|2013-07-25 00:00:...|          256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|        12111|       COMPLETE|
|       4|2013-07-25 00:00:...|         8827|         CLOSED|
|       5|2013-07-25 00:00:...|        11318|       COMPLETE|
|       6|2013-07-25 00:00:...|         7130|       COMPLETE|
|       7|2013-07-25 00:00:...|         4530|       COMPLETE|
|       8|2013-07-25 00:00:...|         2911|     PROCESSING|
|       9|2013-07-25 00:00:...|         5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|         5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|          918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|         1837|         CLOSED|
|      13|2013-07-25 00:00:...|         9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:...|         9842|     PROCESSING|
|      15|2013-07-25 00:00:...|         2568|       COMPLETE|
|      16|2013-07-25 00:00:...|         7276|PENDING_PAYMENT|
|      17|2013-07-25 00:00:...|         2667|       COMPLETE|
|      18|2013-07-25 00:00:...|         1205|         CLOSED|
|      19|2013-07-25 00:00:...|         9488|PENDING_PAYMENT|
|      20|2013-07-25 00:00:...|         9198|     PROCESSING|
+--------+--------------------+-------------+---------------+
only showing top 20 rows

<p style="background:#AED6F1"><b>Reading Text Data from Files</p>

* We can use spark.read.csv or spark.read.text to read text data.
* spark.read.csv can be used for files with some separators between data cols. Default field names will be like _c0, _c1 etc.
* spark.read.text can be used to read fixed length data where there is no delimiter. Default field name is value.
* toDF() function is used to define custom attribute names.
* With either of the above functions for reading, data will be represented as string.
* spark.read.format() is a generic function to use to read the file.
* We can convert data types by using cast function like this -
  <code>df.select(df.field.cast(IntegerType()))</code>

<p style="background:#F1C40F"><b>NOTE: </b> <code>toDF()</code> function doesn't change the datatype of the fields. It just changes their names. </p>

<p style="background :#d0d5db"><b>using spark.read.csv() </p>

<p style="background:#F1C40F"><b>NOTE:  </b> Providing schema while using spark.read.csv allows to provide fields' datatypes along with their custom names.<br>
<code>>>> ordersDF = spark.read.csv('/public/retail_db/orders', sep=',', schema='order_id int, order_date string, cust_id int, status string')</code></p>

In [ ]:
>>> ordersDF = spark.read.csv('/public/retail_db/orders', sep=',', schema='order_id int, order_date string, cust_id int, status string')

In [ ]:
>>> ordersDF.printSchema()
root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- cust_id: integer (nullable = true)
 |-- status: string (nullable = true)

In [ ]:
>>> ordersDF = spark.read.csv('/public/retail_db/orders').toDF('order_id', 'order_date', 'order_cust_id', 'order_status')

In [ ]:
>>> ordersDF.printSchema()                                                      
root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_cust_id: string (nullable = true)
 |-- order_status: string (nullable = true)

<p style="background:#F1C40F"><b>NOTE: </b> Reading Using spark.read.format -><br> If the format is csv, json, orc, parquet, text etc., then load option is to be specified containing path of the file to be read. For reading from jdbc, the load option doesn't need path. </p>

In [ ]:
>>> ordersDF_usingFormat = spark.read.format('csv').option('sep', ',').schema('order_id int, order_date string, cust_id int, status string').load('/public/retail_db/orders')

In [ ]:
>>> ordersDF_usingFormat.printSchema()
root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- cust_id: integer (nullable = true)
 |-- status: string (nullable = true)

In [ ]:
>>> ordersDF_usingFormat.show()
+--------+--------------------+-------+---------------+                         
|order_id|          order_date|cust_id|         status|
+--------+--------------------+-------+---------------+
|       1|2013-07-25 00:00:...|  11599|         CLOSED|
|       2|2013-07-25 00:00:...|    256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|  12111|       COMPLETE|
|       4|2013-07-25 00:00:...|   8827|         CLOSED|
|       5|2013-07-25 00:00:...|  11318|       COMPLETE|
|       6|2013-07-25 00:00:...|   7130|       COMPLETE|
|       7|2013-07-25 00:00:...|   4530|       COMPLETE|
|       8|2013-07-25 00:00:...|   2911|     PROCESSING|
|       9|2013-07-25 00:00:...|   5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|   5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|    918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|   1837|         CLOSED|
|      13|2013-07-25 00:00:...|   9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:...|   9842|     PROCESSING|
|      15|2013-07-25 00:00:...|   2568|       COMPLETE|
|      16|2013-07-25 00:00:...|   7276|PENDING_PAYMENT|
|      17|2013-07-25 00:00:...|   2667|       COMPLETE|
|      18|2013-07-25 00:00:...|   1205|         CLOSED|
|      19|2013-07-25 00:00:...|   9488|PENDING_PAYMENT|
|      20|2013-07-25 00:00:...|   9198|     PROCESSING|
+--------+--------------------+-------+---------------+
only showing top 20 rows

<p style="background:#F1C40F"><b>NOTE: </b> Another way, of specifying the datatype is to use the cast function, as part of select or withColumn and then typecast the fields to their original datatype.</p>

If we have a DF created  as follows :<br>
<code>>>> orders = spark.read.csv('/public/retail_db/orders').toDF('ord_id', 'ord_dt', 'cust_id', 'status')</code>
Now, if we want to select from this DF (select is an API used to project data from data frame, creating a new data frame), we can specify the datatype of fields using these 2 ways - 
* passing datatype as string in cast function .cast("int")<br><code>>>> orders.select(orders.ord_id.cast("int"))
DataFrame[ord_id: int]</code>
* importing types from pyspark.sql.types  such as IntegerType and then oassing them to cast function as below -<br>
<code>>>> from pyspark.sql.types import IntegerType, FloatType</code><br> 
<code>>>> orders.select(orders.ord_id.cast("int"), orders.ord_dt, orders.cust_id.cast(IntegerType()), orders.status)
DataFrame[ord_id: int, ord_dt: string, cust_id: int, status: string]
</code> 

<p style="background:#F1C40F"><b>NOTE: </b> But the above way is an overkill as we may have numerous fields in a given data frame and we may only want to typecast 1 or 2. <br> An alternative approach is to use a function called <b>withColumn()</b>. It takes 2 arguments - the first is the alias for the function or expression we are going to use and the second is the cast function.</p>

<p style="background :#d0d5db">The whole code below -  </p>

In [ ]:
>>> orders = spark.read.csv('/public/retail_db/orders').toDF('order_id', 'order_date', 'customer_id', 'order_status')

>>> orders.printSchema()
root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)

>>> orders.show(5)
+--------+--------------------+-----------+---------------+                     
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
+--------+--------------------+-----------+---------------+
only showing top 5 rows


In [ ]:
>>> #now for casting

>>> from pyspark.sql.types import IntegerType

In [ ]:
>>> ordersDF = orders.withColumn('order_id', orders.order_id.cast("int")). \
... withColumn('customer_id', orders.customer_id.cast(IntegerType()))

>>> ordersDF.printSchema()
root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)

>>> ordersDF.show(5)
+--------+--------------------+-----------+---------------+                     
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
+--------+--------------------+-----------+---------------+
only showing top 5 rows


<p style="background :#d0d5db"><b>using spark.read.text() </p>

In [ ]:
>>> orders = spark.read.text('/public/retail_db/orders')
>>> orders.printSchema()
root
 |-- value: string (nullable = true)

In [ ]:
>>> orders.show(5)
+--------------------+                                                          
|               value|
+--------------------+
|1,2013-07-25 00:0...|
|2,2013-07-25 00:0...|
|3,2013-07-25 00:0...|
|4,2013-07-25 00:0...|
|5,2013-07-25 00:0...|
+--------------------+
only showing top 5 rows

In [ ]:
>>> # to view whole row in the data
... 
>>> orders.show(truncate=False)
+---------------------------------------------+
|value                                        |
+---------------------------------------------+
|1,2013-07-25 00:00:00.0,11599,CLOSED         |
|2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT  |
|3,2013-07-25 00:00:00.0,12111,COMPLETE       |
|4,2013-07-25 00:00:00.0,8827,CLOSED          |
|5,2013-07-25 00:00:00.0,11318,COMPLETE       |
|6,2013-07-25 00:00:00.0,7130,COMPLETE        |
|7,2013-07-25 00:00:00.0,4530,COMPLETE        |
|8,2013-07-25 00:00:00.0,2911,PROCESSING      |
|9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT |
|10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT|
|11,2013-07-25 00:00:00.0,918,PAYMENT_REVIEW  |
|12,2013-07-25 00:00:00.0,1837,CLOSED         |
|13,2013-07-25 00:00:00.0,9149,PENDING_PAYMENT|
|14,2013-07-25 00:00:00.0,9842,PROCESSING     |
|15,2013-07-25 00:00:00.0,2568,COMPLETE       |
|16,2013-07-25 00:00:00.0,7276,PENDING_PAYMENT|
|17,2013-07-25 00:00:00.0,2667,COMPLETE       |
|18,2013-07-25 00:00:00.0,1205,CLOSED         |
|19,2013-07-25 00:00:00.0,9488,PENDING_PAYMENT|
|20,2013-07-25 00:00:00.0,9198,PROCESSING     |
+---------------------------------------------+
only showing top 20 rows


In [ ]:
>>> orders.show(5, truncate=False)
+-------------------------------------------+
|value                                      |
+-------------------------------------------+
|1,2013-07-25 00:00:00.0,11599,CLOSED       |
|2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT|
|3,2013-07-25 00:00:00.0,12111,COMPLETE     |
|4,2013-07-25 00:00:00.0,8827,CLOSED        |
|5,2013-07-25 00:00:00.0,11318,COMPLETE     |
+-------------------------------------------+
only showing top 5 rows
